In [1]:
def add(A, B):
    return A + B

def fancy_func(A, B, C, D):
    E = add(A, B)
    F = add(C, D)
    G = add(E, F)
    return G

fancy_func(1, 2, 3, 4)

10

In [2]:
def add_str():
    return '''
def add(A, B):
    return A + B
'''

def fancy_func_str():
    return '''
def fancy_func(A, B, C, D):
    E = add(A, B)
    F = add(C, D)
    G = add(E, F)
    return G
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))    
'''

prog = evoke_str()
y = compile(prog, '', 'exec')
exec(y)

10


In [3]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(
            nn.Dense(256, activation = 'relu'), 
            nn.Dense(128, activation = 'relu'),
            nn.Dense(2)
        )
    net.initialize()
    return net

x = nd.random.normal(shape = (1, 512))
net = get_net()
net(x)

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL



[[ 0.08827581  0.00505182]]
<NDArray 1x2 @cpu(0)>

In [4]:
net.hybridize()
net(x)


[[ 0.08827581  0.00505182]]
<NDArray 1x2 @cpu(0)>

In [5]:
from time import time
def bench(net, x):
    start = time()
    for i in range(1000):
        y = net(x)
    # wait all calculate finish
    nd.waitall()
    return time() - start

net = get_net()
print('Before hybridizing: %.4f sec' % (bench(net, x)))
net.hybridize()
print('After hybridizing: %.4f sec' % (bench(net, x)))

Before hybridizing: 0.2898 sec
After hybridizing: 0.1131 sec


In [6]:
net.export('my_mlp')

In [7]:
from mxnet import sym

x = sym.var('data')
y = net(x)
y

<Symbol hybridsequential1_dense2_fwd>

In [8]:
class HybridNet(nn.HybridBlock):
    def __init__(self, **kwargs):
        super(HybridNet, self).__init__(**kwargs)
        with self.name_scope():
            self.fc1 = nn.Dense(10)
            self.fc2 = nn.Dense(2)
    
    def hybrid_forward(self, F, x):
        print(F)
        print(x)
        x = F.relu(self.fc1(x))
        print(x)
        return self.fc2(x)

In [9]:
net = HybridNet()
net.initialize()
x = nd.random.normal(shape = (1, 4))
y = net(x)

<module 'mxnet.ndarray' from '/home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/ndarray/__init__.py'>

[[-0.12225834  0.5429998  -0.94693518  0.59643304]]
<NDArray 1x4 @cpu(0)>

[[ 0.11134676  0.04770704  0.05341475  0.          0.08091211  0.          0.
   0.04143535  0.          0.        ]]
<NDArray 1x10 @cpu(0)>


In [10]:
y = net(x)

<module 'mxnet.ndarray' from '/home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/ndarray/__init__.py'>

[[-0.12225834  0.5429998  -0.94693518  0.59643304]]
<NDArray 1x4 @cpu(0)>

[[ 0.11134676  0.04770704  0.05341475  0.          0.08091211  0.          0.
   0.04143535  0.          0.        ]]
<NDArray 1x10 @cpu(0)>


In [11]:
net.hybridize()
y = net(x)

<module 'mxnet.symbol' from '/home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/symbol/__init__.py'>
<Symbol data>
<Symbol hybridnet0_relu0>


In [12]:
y = net(x)

In [13]:
print(y)


[[ 0.00370749  0.00134991]]
<NDArray 1x2 @cpu(0)>
